In [31]:
from langchain_community.utilities import WolframAlphaAPIWrapper
from dotenv import load_dotenv
import os
import requests
import xml.etree.ElementTree as ET

APPID = os.getenv("WOLFRAM_ALPHA_APPID")

base_url = "http://api.wolframalpha.com/v2/query"

In [40]:
def wolfram_query_structured(query: str) -> dict:
    params = {
        "input": query, 
        "appid": APPID, 
        "format": "plaintext"
    }

    resp = requests.get(base_url, params=params)
    resp.raise_for_status()

    root = ET.fromstring(resp.text)
    results = {}
    for pod in root.findall(".//pod"):
        title = pod.attrib.get("title")
        subpods = []
        for sub in pod.findall("subpod"):
            plain = sub.find("plaintext")
            if plain is not None and plain.text:
                subpods.append(plain.text.strip())
        if subpods:
            results[title] = subpods if len(subpods) > 1 else subpods[0]
    return results

print(wolfram_query_structured("What is e^x?"))


{'Input': 'e^x', 'Roots': '(no roots exist)', 'Properties as a real function': ['R (all real numbers)', '{y element R : y>0} (all positive real numbers)', 'injective (one-to-one)'], 'Periodicity': 'periodic in x with period 2 i π', 'Series expansion at x = 0': '1 + x + x^2/2 + x^3/6 + x^4/24 + O(x^5)\n(Taylor series)', 'Derivative': 'd/dx(e^x) = e^x', 'Indefinite integral': 'integral e^x dx = e^x + constant', 'Limit': 'lim_(x->-∞) e^x = 0', 'Alternative representation': ['e^x = z^x for z = e', 'e^x = w^a for a = x/log(w)', 'e^x = 1 + 2/(-1 + coth(x/2))'], 'Series representation': ['e^x = sum_(k=0)^∞ x^k/(k!)', 'e^x = sum_(k=-∞)^∞ I_k(x)', 'e^x = sum_(k=0)^∞ (x^(-1 + 2 k) (2 k + x))/((2 k)!)'], 'Integral representation': '(1 + z)^a = ( integral_(-i ∞ + γ)^(i ∞ + γ) (Γ(s) Γ(-a - s))/z^s ds)/((2 π i) Γ(-a)) for (0<γ<-Re(a) and abs(arg(z))<π)', 'Definite integral': 'integral_(-∞)^0 e^x dx = 1', 'Definite integral over a half-period': 'integral_0^(i π) e^x dx = -2'}
